Covariant shift detection for tabular data 

This example of feature drift detection for tabular data is inspired by
https://docs.seldon.io/projects/alibi-detect/en/latest/examples/cd_online_wine.html


In [1]:
import sys 
sys.path.append("../../") # Adding path to project root for this specific notebook

In [6]:
import pandas as pd
import numpy as np

red = pd.read_csv(
    "https://storage.googleapis.com/seldon-datasets/wine_quality/winequality-red.csv", sep=';'
)
white = pd.read_csv(
    "https://storage.googleapis.com/seldon-datasets/wine_quality/winequality-white.csv", sep=';'
)
white = white.drop(["quality"], axis=1)
red = red.drop(["quality"], axis=1)
white = white[:500].astype(np.float64)
red = red[:500].astype(np.float64)



In [7]:

from drifting import DriftingClient, DriftType

client = DriftingClient()
detector_name = "WineDriftDetector"

client.fit(white, drift_type=DriftType.TABULAR, detector_name=detector_name)

<Response [200]>

In [8]:
client.load(detector_name=detector_name)

<Response [200]>

In [12]:
red_white_red = pd.concat([red, white, red], axis=0)
drifts = []
for i in range(len(red_white_red)):
    is_drift, stat_val = client.predict(red_white_red.iloc[[i], :], drift_type=DriftType.TABULAR, detector_name=detector_name)
    # print(response)
    if is_drift:
        # The drift detector indicates after each sample if there is a drift in the data
        print(f'Change detected at index {i}')
        drifts.append(i)
    break


In [10]:
is_drift

0